# Basic 3-asset portfolio plots

In [5]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import minimize

##### Inputs
# Risk-free rate
r = 0.02
# Expected returns
mns = np.array([0.10, 0.05, 0.07])
# Standard deviations
sds = np.array([0.20, 0.12, 0.15])
# Correlations
corr12 = 0.3
corr13 = 0.3
corr23 = 0.3
# Covariance matrix
C  = np.identity(3)
C[0, 1] = C[1, 0] = corr12
C[0, 2] = C[2, 0] = corr13
C[1, 2] = C[2, 1] = corr23
cov = np.diag(sds) @ C @ np.diag(sds)

##### Tangency Portfolio
w = np.linalg.solve(cov, mns - r)
wgts_tangency = w / np.sum(w)
mnTang = mns @ wgts_tangency
sdTang = np.sqrt(wgts_tangency @ cov @ wgts_tangency)
srTang = (mnTang - r)/sdTang

##### Frontier Portfolios
def frontier(means, cov, target):
    n = len(means)
    w = np.linalg.solve(cov, np.ones(n))
    wgts_gmv = w / np.sum(w)
    w = np.linalg.solve(cov, means)
    piMu = w / np.sum(w)
    m1 = wgts_gmv @ means
    m2 = piMu @ means
    a = (target - m2) / (m1 - m2)
    wgts = a * wgts_gmv + (1 - a) * piMu
    return wgts
wgts_frontier = [frontier(mns, cov, m) for m in np.linspace(mns.min(), mns.max(),40)]
mn_frontier = [mns @ w for w in wgts_frontier]
sd_frontier = [np.sqrt(w @ cov @ w) for w in wgts_frontier]

##### CAL
wgt_cal = np.arange(0, 1.5, 0.05)
mn_cal = [w*mnTang + (1-w)*r for w in wgt_cal]
sd_cal = [w*sdTang for w in wgt_cal]


In [11]:
##### Plot portfolios in expected return-stdev space
fig = go.Figure()  

# Plot assets
for i in np.arange(len(mns)):
    trace = go.Scatter(x=[sds[i]], y=[mns[i]], mode="markers", hovertemplate="Asset "+str(i+1),marker=dict(size=10, color="red"), name="Asset "+str(i+1)) 
    fig.add_trace(trace)

# Plot frontier
string = "<br>"
string += "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "<extra></extra>"
trace0 = go.Scatter(x=sd_frontier, y=mn_frontier, mode="lines", customdata=wgts_frontier, hovertemplate=string, marker=dict(size=10, color="blue"), name="Frontier")       
fig.add_trace(trace0)

# Plot tangency
string = "<br>Tangency Portfolio:<br>"
string += "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "<extra></extra>"
trace1 = go.Scatter(x=[sdTang], y=[mnTang], mode="markers", customdata=[wgts_tangency], hovertemplate=string, marker=dict(size=10, color="blue"), name="Tangency")   
fig.add_trace(trace1)

# Plot CAL
string = "<br>"
string += "Weight in Tangency: %{customdata:.1%}<br>"
string += "Weight in Risk-free:  %{text:.1%}<br>"
string += "<extra></extra>"
trace2 = go.Scatter(x=sd_cal, y=mn_cal, mode="lines", text = 1-wgt_cal,customdata=wgt_cal, hovertemplate=string, marker=dict(size=10, color="black"), name="CAL")       
fig.add_trace(trace2)

fig.layout.xaxis["title"] = "Standard Deviation"
fig.layout.yaxis["title"] = "Expected Return"
fig.update_xaxes(range=[0, 1.05 * sds.max()])
fig.update_yaxes(range=[0, 1.05 * mns.max()])
fig.update_yaxes(tickformat=".0%")
fig.update_xaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.95, xanchor="left", x=0.05))
fig.show()

In [104]:
# Simulate return realizations
from scipy.stats import multivariate_normal as mvn

# Periods
T = 30


# Realizations
def returns(mns, cov, init_wgts, T, seed):
    realized = mvn.rvs(mns, cov, size=T,random_state=seed)


    ret_cols = ['ret' + str(i) for i in range(len(mns))] 
    beg_wgt_cols = ['beg_wgt' + str(i) for i in range(len(mns))] 
    end_wgt_cols = ['end_wgt' + str(i) for i in range(len(mns))] 
    df = pd.DataFrame(dtype=float, columns=beg_wgt_cols + ret_cols + end_wgt_cols + ['exp_pret','sd_pret'], index=np.arange(T)+1)
    df[ret_cols] = realized
    # print(df)
    df.loc[1,beg_wgt_cols] = init_wgts
    for t in df.index:
        if t > 1:
            df.loc[t,beg_wgt_cols] = df.loc[t-1,end_wgt_cols].values
        wgts = df.loc[t,beg_wgt_cols].values
        # print(wgts)
        rets = df.loc[t,ret_cols].values
        # print(rets)
        rp = wgts @ rets
        # print(rp)
        for i, w in enumerate(end_wgt_cols):
            df.loc[t,w] = wgts[i] * (1+rets[i]) / (1+rp)
        df.loc[t,'exp_pret'] = wgts @ mns
        df.loc[t,'sd_pret']  = np.sqrt(wgts @ cov @ wgts)        
    return df


rets = returns(mns, cov, wgts_tangency, T, 26)
rets['sharpe ratio'] = (rets.exp_pret - r) / rets.sd_pret


In [105]:

##### Plot non-rebalanced portfolios in expected return-stdev space
fig = go.Figure()  

# Plot assets
for i in np.arange(len(mns)):
    trace = go.Scatter(x=[sds[i]], y=[mns[i]], mode="markers", hovertemplate="Asset "+str(i+1),marker=dict(size=10, color="red"), name="Asset "+str(i+1)) 
    fig.add_trace(trace)

# Plot frontier
string = "<br>"
string += "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "<extra></extra>"
trace0 = go.Scatter(x=sd_frontier, y=mn_frontier, mode="lines", customdata=wgts_frontier, hovertemplate=string, marker=dict(size=10, color="blue"), name="Frontier")       
fig.add_trace(trace0)

# Plot tangency
string = "<br>Tangency Portfolio:<br>"
string += "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "Sharpe ratio: %{text:.4f}<br>"
string += "<extra></extra>"
trace1 = go.Scatter(x=[sdTang], y=[mnTang], mode="markers", text = [srTang], customdata=[wgts_tangency], hovertemplate=string, marker=dict(size=10, color="blue"), name="Tangency")   
fig.add_trace(trace1)

# Plot first five-years of non-rebalanced portfolios
string = "<br>Year %[x:.0f}<br>"
string += "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "Sharpe ratio: %{customdata[3]:.4f}<br>"
string += "<extra></extra>"
cd_cols = ['beg_wgt' + str(i) for i in range(len(mns))]+['sharpe ratio']
for i in rets.index[1:5]:
    trace = go.Scatter(x=[rets.loc[i,'sd_pret']], y=[rets.loc[i,'exp_pret']], mode="markers", 
        customdata=rets.loc[i,cd_cols], hovertemplate="Year "+str(i),marker=dict(size=10, color="black"), name="Year "+str(i)) 
    fig.add_trace(trace)

fig.layout.xaxis["title"] = "Standard Deviation"
fig.layout.yaxis["title"] = "Expected Return"
fig.update_xaxes(range=[0.7*sds.min(), 1.05*sds.max()])
fig.update_yaxes(range=[0.7*mns.min(), 1.05*mns.max()])
fig.update_yaxes(tickformat=".0%")
fig.update_xaxes(tickformat=".0%")
fig.update_layout(legend=dict(yanchor="top", y =0.95, xanchor="left", x=0.05))
fig.show()

In [106]:
# Animated version of picture above
stringFront = "<br>"
stringFront += "Asset 1: %{customdata[0]:.1%}<br>"
stringFront += "Asset 2: %{customdata[1]:.1%}<br>"
stringFront += "Asset 3: %{customdata[2]:.1%}<br>"
stringFront += "<extra></extra>"

stringTang = "<br>Tangency Portfolio:<br>"
stringTang += "Asset 1: %{customdata[0]:.1%}<br>"
stringTang += "Asset 2: %{customdata[1]:.1%}<br>"
stringTang += "Asset 3: %{customdata[2]:.1%}<br>"
stringTang += "Sharpe ratio: %{text:.4f}<br>"
stringTang += "<extra></extra>"

string = "<br>Year %[x:.0f}<br>"
string += "Asset 1: %{customdata[0]:.1%}<br>"
string += "Asset 2: %{customdata[1]:.1%}<br>"
string += "Asset 3: %{customdata[2]:.1%}<br>"
string += "Sharpe ratio: %{customdata[3]:.4f}<br>"
string += "<extra></extra>"


fig = go.Figure(
    data=[go.Scatter(x=sd_frontier, y=mn_frontier, mode="lines", customdata=wgts_frontier, hovertemplate=stringFront, marker=dict(size=10, color="blue")),
          go.Scatter(x=sd_frontier, y=mn_frontier, mode="lines", customdata=wgts_frontier, hovertemplate=stringFront, marker=dict(size=10, color="blue")),
          go.Scatter(x=[sdTang], y=[mnTang], mode="markers", text = [srTang], customdata=[wgts_tangency], hovertemplate=stringTang, marker=dict(size=10, color="blue"))   
          ]+ 
          [go.Scatter(x=[sds[k]], y=[mns[k]], mode="markers", hovertemplate="Asset "+str(k+1)+ "<extra></extra>",
            marker=dict(size=10, color="red")) for k in np.arange(len(mns))],
    layout=go.Layout(
        xaxis=dict(range=[0.7*sds.min(), 1.05*sds.max()], autorange=False, zeroline=False),
        yaxis=dict(range=[0.7*mns.min(), 1.05*mns.max()], autorange=False, zeroline=False),
        hovermode="closest",showlegend=False,
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None,{"frame": {"duration": 800, "redraw": False},"fromcurrent": True}]),
                                #    dict(label="Pause",
                                #         method="animate",
                                #         args=[None,{"frame": {"duration": 0, "redraw": False},"mode": "immediate","transition": {"duration": 0}}]) 
                                        ])]),
    frames=[go.Frame(
        data=[go.Scatter(x=[rets.loc[i,'sd_pret']],y=[rets.loc[i,'exp_pret']],mode="markers",hovertemplate="Year "+str(i) + "<extra></extra>",marker=dict(size=10, color="black")),
            ],
        layout=go.Layout(annotations=[go.layout.Annotation(x=sdTang, y=1.1*mnTang, xref="x", yref="y",
            text="Year " + str(i) +"<br>Sharpe ratio: " +str(np.round(rets.loc[i,'sharpe ratio'],4)),
            showarrow=False, )])) for i in rets.index]
)

fig.show()